<a href="https://colab.research.google.com/github/2of/Data301---Big-Data-Computing-University-of-Canterbury-/blob/main/GDELT_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

DATA301 Assignment

In [ ]:
WORKING_DIR = "/content/drive/My Drive/Data301Assignment2020"
ARTICLES = WORKING_DIR + "/ARTICLES"

In [ ]:
!ls

drive  sample_data


In [ ]:
''' Setup ''' 
#library and code setup
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!pip install -q pyspark
!pip install wget
!pip install newspaper3k
!pip install gdelt
import pyspark, os, sys

os.environ["JAVA_HOME"]="/usr/lib/jvm/java-8-openjdk-amd64/"

#start spark local server
from operator import add
import time

os.environ["PYSPARK_PYTHON"]="python3"

import pyspark
from pyspark import SparkConf, SparkContext

#connects our python driver to a local Spark JVM running on the Google Colab server virtual machine
try:
  conf = SparkConf().setMaster("local[*]").set("spark.executor.memory", "1g")
  sc = SparkContext(conf = conf)
except ValueError:
  #it's ok if the server is already started
  pass

def dbg(x):
  """ A helper function to print debugging information on RDDs """
  if isinstance(x, pyspark.RDD):
    print([(t[0], list(t[1]) if 
            isinstance(t[1], pyspark.resultiterable.ResultIterable) else t[1])
           if isinstance(t, tuple) else t
           for t in x.take(100)])
  else:
    print(x)

Tool to convert country list to relevant actor1_id in gdelt (returns three letter codes from whatever is in countries

In [ ]:
 !pip install pycountry
import pycountry

countries= """Argentina
Australia
Belgium
China
Colombia
Czech Republic
Denmark
El Salvador
France
Germany
India
Israel
Italy
Japan
Jordan
Kenya
Kuwait
Malaysia
Morocco
New Zealand
Norway
Poland
Republic of Ireland
Russia
Saudi Arabia
South Africa
Spain
UK"""


mapping = {country.name: country.alpha_3 for country in pycountry.countries}
country_codes = {mapping.get(a) for a in countries.splitlines()}
country_codes.remove(None)
set_a = list(country_codes)

print(set_a)


['CHN', 'AUS', 'MAR', 'NZL', 'ZAF', 'NOR', 'COL', 'DNK', 'BEL', 'FRA', 'ISR', 'MYS', 'ITA', 'JOR', 'JPN', 'SLV', 'ESP', 'DEU', 'SAU', 'KWT', 'IND', 'ARG', 'POL', 'KEN']


Other Countries in GDELT (i.e. no lockdown)

In [ ]:
# Set the working directory this way
%cd content
%cd drive/My\ Drive/Data301Assignment2020/

# # ! rm *
# %cd ARTICLES
# !ls
# !rm *
# %cd .. 

# !ls
# # !ls



[Errno 2] No such file or directory: 'content'
/content
/content/drive/My Drive/Data301Assignment2020


In [ ]:
main_list = ['CHN', 'AUS', 'BEL', 'CAN', 'USA', 'NZL', 'LKA', 'RUS', 'TUR', 'UZB', 'ZWE', 'AFG', 'ARG', 'AZE', 'BEN', 'BGD', 'BMU', 'BRA', 'BRN', 'CHE', 'CHL', 'COL', 'CUB', 'CYP', 'DEU', 'DNK', 'EGY', 'ESP', 'EUR', 'FRA', 'GBR', 'GHA', 'GRC', 'GUY', 'IDN', 'WST', 'IND', 'IRL', 'IRN', 'IRQ', 'ISL', 'ISR', 'JAM', 'JOR', 'JPN', 'KAZ', 'KEN', 'KOR', 'LBN', 'LBY', 'MEX', 'MKD', 'MMR', 'MWI', 'MYS', 'NGA', 'NLD', 'PER', 'PHL', 'POL', 'PRK', 'PRT', 'PSE', 'QAT', 'SAU', 'SGP', 'SWE', 'SYR', 'THA', 'TKM', 'TWN', 'TZA', 'VEN', 'VNM', 'YEM', 'ZAF']

import numpy as np
no_unrest = np.setdiff1d(main_list,set_a) # may also contain error country codes, coutnry codes that GDELT doesnt care about
print(no_unrest,len(no_unrest))

['AFG' 'AZE' 'BEN' 'BGD' 'BMU' 'BRA' 'BRN' 'CAN' 'CHE' 'CHL' 'CUB' 'CYP'
 'EGY' 'EUR' 'GBR' 'GHA' 'GRC' 'GUY' 'IDN' 'IRL' 'IRN' 'IRQ' 'ISL' 'JAM'
 'KAZ' 'KOR' 'LBN' 'LBY' 'LKA' 'MEX' 'MKD' 'MMR' 'MWI' 'NGA' 'NLD' 'PER'
 'PHL' 'PRK' 'PRT' 'PSE' 'QAT' 'RUS' 'SGP' 'SWE' 'SYR' 'THA' 'TKM' 'TUR'
 'TWN' 'TZA' 'USA' 'UZB' 'VEN' 'VNM' 'WST' 'YEM' 'ZWE'] 57


Create Files


In [ ]:
upper_date_bound = "2020 April 14"
lower_date_bound = "2020 April 1"
!pip install wget
from concurrent.futures import ProcessPoolExecutor
from datetime import date, timedelta
import pandas as pd
import gdelt
import os

# set up gdeltpyr for version 2
gd = gdelt.gdelt(version=2)

# multiprocess the query
e = ProcessPoolExecutor()


# generic functions to pull and write data to disk based on date
def get_filename(x):
  date = x.strftime('%Y%m%d')
  return "{}.csv".format(date)

def intofile(filename):
    try:
        if not os.path.exists(filename):
          date = filename.split(".")[0]
          d = gd.Search(date, table='events',coverage=False) #not updata at 15mins
          d.to_csv(filename,encoding='utf-8',index=False)
    except:
        print("Error occurred")

# pull the data from gdelt into multi files; this may take a long time
dates = [get_filename(x) for x in pd.date_range(lower_date_bound,upper_date_bound)]


results = list(e.map(intofile,dates))


In [ ]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)


data_urls = sqlContext.read.option("header", "true").csv(WORKING_DIR + "/*.csv")
df = data_urls.select('Actor1CountryCode',"EVENTCODE","SOURCEURL","AvgTone")

file_info = df.rdd.map(lambda x: ((x[0],x[1],x[3]),x[2]))
file_info = file_info.filter(lambda x: (x[0][0] != None))




#Categorization

In [ ]:
import itertools,operator
import numpy as np

%cd KEYWORDS/
!ls 


keys = ['healthcare','laws','politics','police','economy','education','international_relations','general_public_interest']
def make_big_dictionary():
  files = ['economy','healthcare','economy','laws','police','politics','welfare']
  model = {}
  
  for f in files: 
    try:
      a = open(f+".txt").readlines()
      for k in a:
        model[k.replace('\n','')] = f
    except:
      pass

  return model
print(make_big_dictionary())



def convert_to_dictionary():
  '''The keywords are categorized as a dictionary instead of having to lookup in each one, should yield performance improvements'''
  model = {}
  healthcare = {'doctor','hospital','medical','health','nurse','equipment','dying','patients','health'}
  laws = {'legistlation','referendum','legal','laws'}
  politics = {"leader",'majority','opposition'}
  police = {'laws','secutity','protest','unrest',}
  economy = {'fiancial','security','ratepayers','economic'}
  education = {'school','teacher','education','learning'}
  international_relations = {'emmigration','immigration','international','relations'}
  general_public_interest ={'work','family'}
  all_sets = [healthcare,laws,politics,police,economy,education,international_relations,general_public_interest]
  keys = ['healthcare','laws','politics','police','economy','education','international_relations','general_public_interest']

  for i in range(0,len(all_sets)):
    candidate = list(all_sets[i])
    for keyword in candidate:
      model[keyword] = keys[i]


  return model

b = ['hungarys', 'charlemagne', 'hungarian', 'viktor', 'parties', 'european', 'orban', 'gets', 'away', 'mr', 'legal', 'leader', 'eu', 'majority', 'opposition', 'orbans']
b = (('HUN', '051', '-3.66541353383459'), ['hungarys', 'charlemagne', 'hungarian', 'viktor', 'parties', 'european', 'orban', 'gets', 'away', 'mr', 'legal', 'leader', 'eu', 'majority', 'opposition', 'orbans'])

def make_relevance_vector(a):
  relevance = {key:0 for key in keys}
  for each in a: 
    try:
      match = model[each]
      relevance[match] = relevance[match] + 1
    except:
      pass
  return np.array(list(relevance.values()))


def protest_to_dict():
  model = {}
  try: 
    a = open("protest.txt").readlines()
    return set([k.replace('\n','') for k in a])
  except:
    return {}



def make_relevance_vector_protest(a):
  relevance = 0
  for each in a: 
    if each in protmodel:
      relevance += 1
  return np.array([relevance,1])



  bb = list(relevance.values())
  bb.append(1) #the final value used for counts
  return np.array(bb)
  
model = make_big_dictionary()
protmodel = protest_to_dict()
print(protmodel)



print (make_relevance_vector(b))
def in_place_group_by_key(l):
    it = itertools.groupby(l, operator.itemgetter(0))
    for key, subiter in it:
       yield key, sum(item[1] for item in subiter) 



[Errno 2] No such file or directory: 'KEYWORDS/'
/content/drive/My Drive/Data301Assignment2020/KEYWORDS
economy.txt	 health.txt  out.csv	 politics.txt  welfare.txt
healthcarewords  laws.txt    police.txt  protest.txt
{'economics': 'welfare', 'economic': 'economy', 'saving': 'economy', 'sector': 'economy', 'demand': 'economy', 'currency': 'economy', 'inflation': 'economy', 'finance': 'economy', 'goods': 'economy', 'financial': 'economy', 'gdp': 'economy', 'trade': 'economy', 'production': 'economy', 'consumption': 'economy', 'thriftiness': 'economy', 'barter': 'economy', 'distribution': 'economy', 'economic system': 'economy', 'market economy': 'economy', 'service': 'economy', 'good': 'welfare', 'growth': 'economy', 'recession': 'economy', 'slowing': 'economy', 'recovery': 'economy', 'market': 'economy', 'downturn': 'economy', 'slowdown': 'economy', 'unemployment': 'welfare', 'crisis': 'economy', 'economist': 'economy', 'markets': 'economy', 'global': 'economy', 'consumer': 'economy', '

READ csv's and download their contents: Getting Keywords and eventually writing these out to a file somewhere. Google cloud storage be damned!


In [ ]:
ARTICLES = WORKING_DIR + "/ARTICLES"
import numpy as np
from newspaper import Article
import nltk
nltk.download('popular', quiet=True)



if not os.path.exists(ARTICLES):
  os.mkdir(ARTICLES)
import wget

def get_download_filename(name,country,importance):
  imp = str(importance)[0:4]
  return ARTICLES + "/" + country + "_" + imp +"_" + name +".html"
def extract_url(line):
  return line
def download_article(line):
  return line[0][1]
def download_url(line):

  country = line[0][0][0]
  importance = line[0][0][2]
  url = line[0][1]
  id_ = str(line[1])
  filename = get_download_filename(id_,country,importance)

  try:
    if not os.path.exists(filename):
      wget.download(url,filename)
    return(filename)
  except:
    return "ERROR"


def scrape_keywords(row):
  try:
    filename =  (row[1])
    filenameurl = "file://"+filename
    a = open(filename)
    art = Article(filenameurl)
    with open(filename, 'r') as f:
      s = f.read()
      art.set_html(s)
    art.parse()
    art.nlp()
    return [(k) for k in art.keywords]
  except:
    return ['']


dd = file_info.zipWithUniqueId()

#-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_#
'''RESTRICT SIZE SO THAT GOOGLE COLLAB IS USABLE '''
#-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_#
# test = sc.parallelize(dd.take(25))
# dbg(test)

# test = test.map(lambda x: (((x[0][0],x[0][1]),download_url(x))))

# dbg(test)
# test = test.map(lambda x: ((x[0][0],scrape_keywords(x))))
# test3 = test.map(lambda x: (x[0][0], make_relevance_vector(x)))

# test3 = test2.reduceByKey(lambda a,b: np.add(a, b))







print("DONE")
#-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_#
'''RESTRICT SIZE SO THAT GOOGLE COLLAB IS USABLE '''
#-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_#


#These are in an if False because they are heavy operations; precaution against accidental execution

if False:
  dd.map(lambda x: ((x[0][0],download_url(x))))
  dd = dd.map(lambda x: (((x[0][0],x[0][1]),download_url(x))))
  dd = dd.collect()




DONE


In [ ]:
%cd ..


/content/drive/My Drive/Data301Assignment2020


In [ ]:
import os
!ls
def extract_country(string):
  try:
    a = string.split("_")
    return ((a[0],a[1]),string)
  except:
    return string

  
def scrape_keywords2(url):
  filename =  ARTICLES + "/" + url
  filenameurl = "file://"+filename
 
  try:
    a = open(filename)
    art = Article(filenameurl)
    with open(filename, 'r') as f:
      s = f.read()
      art.set_html(s)
      art.parse()
      art.nlp()
      return [(k) for k in art.keywords]
  except:
      return []





%cd ARTICLES
files = sc.parallelize([f for f in os.listdir('.') if os.path.isfile(f)])
files = files.map(extract_country)

dbg(files)
files = files.map(lambda x : (x[0] , scrape_keywords2(x[1])))


dbg(files)




20200401.csv  20200405.csv  20200409.csv  20200413.csv	   KEYWORDS
20200402.csv  20200406.csv  20200410.csv  20200414.csv
20200403.csv  20200407.csv  20200411.csv  all_vectors.txt
20200404.csv  20200408.csv  20200412.csv  ARTICLES
/content/drive/My Drive/Data301Assignment2020/ARTICLES
[(('CAN', '-1.3'), 'CAN_-1.3_8854.html'), (('CAN', '-0.9'), 'CAN_-0.9_8856.html'), (('CAN', '-4.3'), 'CAN_-4.3_8858.html'), (('CAN', '0.24'), 'CAN_0.24_8860.html'), (('CAN', '0.24'), 'CAN_0.24_8862.html'), (('CAN', '0.24'), 'CAN_0.24_8864.html'), (('CAN', '-2.9'), 'CAN_-2.9_8866.html'), (('CAN', '0.24'), 'CAN_0.24_8868.html'), (('CAN', '0.24'), 'CAN_0.24_8870.html'), (('CAN', '0.24'), 'CAN_0.24_8872.html'), (('CAN', '0.24'), 'CAN_0.24_8874.html'), (('CAN', '0.24'), 'CAN_0.24_8876.html'), (('CAN', '0.24'), 'CAN_0.24_8878.html'), (('CAN', '0.24'), 'CAN_0.24_8880.html'), (('CAN', '0.24'), 'CAN_0.24_8882.html'), (('CAN', '0.24'), 'CAN_0.24_8884.html'), (('CAN', '-0.9'), 'CAN_-0.9_8886.html'), (('CAN', '-1.3')

Flatten out and write our keywords to a csv file (this keeps it modular. There's a fair bit of overhead but. Whatever!)

In [ ]:
def flat_list(x):
  try:
    return [x[0][0],x[0][1]] + x[1]
  except:
    return []



if False: # if false to prevent calling if new runtime
  dbg(files)

  flattenrdd = files.map(lambda a: (flat_list(a)))
  all_rows = flattenrdd.collect()


  dbg(flattenrdd)



  a = (('CHN', '-3.2'), ['ban', 'components', 'american', 'huawei', 'ceramic', 'trump', 'phones', 'smartphone', 'nearly', 'p40', 'tech', 'using', 'flagship'])


  import csv
  with open("out.csv", "w", newline="") as f:
      writer = csv.writer(f)
      writer.writerows(all_rows)

In [ ]:
%cd .. 
!pwd

/content/drive/My Drive/Data301Assignment2020
/content/drive/My Drive/Data301Assignment2020


In [ ]:
from pyspark.sql import SQLContext
import numpy as np
import math
sqlContext = SQLContext(sc)
!pwd
# %cd .. 

a = sqlContext.read.option("header", "true").csv(WORKING_DIR + "/KEYWORDS" + "/out.csv")
print(a)
raw_words = a.rdd.map(lambda x: (x[0:2], list(x[2:])))

data_urls = raw_words.map(lambda x: (x[0],make_relevance_vector(x[1])))
dbg(data_urls)

# SPLIT #
def safe_to_float(x):
  try:
    return float(x)
  except:
    return 0
data_tones = data_urls.map(lambda x: (x[0][0],safe_to_float(x[0][1]))).reduceByKey(lambda a,b: a + b)
total_countries = data_tones.collect()
# Gives total tone (it's not perfect) #


data_urls = data_urls.map(lambda x: (x[0][0],x[1]))
data_urls = data_urls.reduceByKey(lambda a,b: np.add(a,b))
final = data_urls.collect()



''' Takes the average mean avg. tone (only 144 possible keys so even doing this on the root
will be fine)
'''
def compute_avg_tone(tone,count):
  return tone / count
jointones = data_tones.join(data_urls)
average_tones = jointones.map(lambda x: (x[0], compute_avg_tone(x[1][0], x[1][1][-1]), x[1][1]))


def exchange_value(array, val, which): 
  '''Rudimentary normalization'''
  a = array.tolist()
  try:
    a[-1] = a[-1]/val
  except:
    pass
  summ = math.sqrt(sum([i**2 for i in array]))
  if (summ == 0):
    return [(1/(1))*j for j in a ]
  return [(1/(summ))*j for j in a ]



adjustedarray = jointones.map(lambda x: (x[0],exchange_value(x[1][1],x[1][0],-1)))
dbg(adjustedarray)



final_vectors = adjustedarray.collect()
print(final_vectors)
print(len(final_vectors))


USA_VEC = []
for each in final_vectors:
  if each[0] == 'USA':
    USA_VEC = np.array(each[1])

print(USA_VEC)



/content/drive/My Drive/Data301Assignment2020/KEYWORDS
DataFrame[CAN: string, 0.24: string, doordash: string, program: string, company: string, million: string, cut: string, local: string, 50: string, commission: string, restaurants: string, pay: string, reduced: string, restaurant: string, end: string, commissions: string, partners: string]
[(('CAN', '-1.3'), array([0, 0, 0, 0, 1, 0, 0, 0])), (('CAN', '-4.4'), array([0, 0, 0, 0, 0, 0, 0, 0])), (('CAN', '-1.3'), array([0, 0, 0, 0, 1, 0, 0, 0])), (('CAN', '-2.9'), array([0, 2, 1, 0, 0, 0, 0, 0])), (('CAN', '-10.'), array([0, 0, 0, 2, 0, 0, 0, 0])), (('CAN', '-4.9'), array([0, 0, 1, 2, 0, 0, 0, 0])), (('CAN', '-6.2'), array([0, 0, 0, 1, 1, 0, 0, 0])), (('CAN', '-4.3'), array([0, 0, 0, 0, 0, 0, 0, 0])), (('CAN', '-8.4'), array([0, 0, 0, 4, 0, 0, 0, 0])), (('CAN', '-4.4'), array([0, 0, 0, 0, 0, 0, 0, 0])), (('CAN', '-2.2'), array([0, 0, 0, 1, 0, 0, 0, 0])), (('CAN', '-1.8'), array([0, 1, 1, 0, 0, 0, 0, 0])), (('CAN', '-4.1'), array([0, 0, 

# Cosine similarity 
(output is simply the most similar overall, including the most similar article tone)

In [ ]:
import math


def cos_similarity(v1, v2):
  return v1.dot(v2) / (np.linalg.norm(v1)*np.linalg.norm(v2))
b = adjustedarray.map(lambda x: (x[0],np.array(x[1])))
c = b.map(lambda x: (x[0],cos_similarity(x[1], USA_VEC),x[1]))
top_similar = c.takeOrdered(6, key = lambda x: 1-x[1])

print ("The most similar countries by media outlet  to the USA including tone")
for n in top_similar:
  print(n[0],n[1], list(n[2]))




e = b.map(lambda x: (x[0],cos_similarity(x[1][1:-1], USA_VEC[1:-1]),x[1]))
top_similar_just_media = e.takeOrdered(6, key = lambda x: 1-x[1])

print ("\nThe most similar countries by media outlet  to the USA ONLY articles")
for n in top_similar_just_media:
  print(n[0],n[1], list(n[2]))

def simple_distance(x1,x2):
  return abs(x1 - x2)
d = b.map(lambda x: (x[0],simple_distance(x[1][-1], USA_VEC[-1]),x[1]))
top_similar_tone = d.takeOrdered(6, key = lambda x: x[1])
print ("\nThe most similar countries by overall tone to the USA")
for n in top_similar_tone:
  print(n[0],n[1], list(n[2]))
 
top_similar_tone = d.takeOrdered(6, key = lambda x: 1-x[1])
print ("\nThe least similar countries by overall tone to the USA")
for n in top_similar_tone:
  print(n[0],n[1], list(n[2]))

The most similar countries by media outlet  to the USA including tone
USA 1.0 [0.0, 0.2504455418419026, 0.38616603008556843, 0.6801021151215739, 0.5706259201846943, 0.0, 0.0, -0.0]
AUS 0.9966460436747704 [0.0, 0.19252286344763223, 0.44005225930887365, 0.6600783889633105, 0.5775685903428966, 0.0, 0.0, -0.0]
CAN 0.9963715306026975 [0.0, 0.20880991222872944, 0.4027048307268354, 0.6338872335515001, 0.6264297366861883, 0.0, 0.0, -0.0]
GRC 0.9959538361722424 [0.0, 0.18257418583505536, 0.3651483716701107, 0.7302967433402214, 0.5477225575051661, 0.0, 0.0, -0.0]
HRV 0.9923441202101517 [0.0, 0.31622776601683794, 0.31622776601683794, 0.6324555320336759, 0.6324555320336759, 0.0, 0.0, -0.0]
KOR 0.9840423634541837 [0.0, 0.0890870806374748, 0.44543540318737396, 0.7126966450997984, 0.5345224838248488, 0.0, 0.0, -0.0]

The most similar countries by media outlet  to the USA ONLY articles
USA 1.0 [0.0, 0.2504455418419026, 0.38616603008556843, 0.6801021151215739, 0.5706259201846943, 0.0, 0.0, -0.0]
AUS 0.

**UNREST** (countries who show much and show little unrest) *note: we only consider internal transcripts

In [ ]:
''' follows the same as categorizing other data'''
dbg(raw_words)
test = raw_words.map(lambda x: x[0][1])

def safe(x):
  try:
    return float(x)
  except:
    return 0

protests = raw_words.map(lambda x: ((x[0][0], safe(x[0][1])),x[1]))
protests = protests.filter(lambda x: x[0][1] <= 0).map(lambda x: (x[0][0],make_relevance_vector_protest(x[1]))).reduceByKey(lambda a,b: np.add(a,b)).map(lambda x: (x[0],(x[1][0]/x[1][1])*100)) 
#ONLY conisder negative toned articles abount unrest. Then take Percentage of total articles per country as the metric for how much unrest.

protests = protests.filter(lambda x: x[1] > 0)
most_protests = protests.takeOrdered(40, key = lambda x: 1-x[1])
print("Below are the countries and their negative article percentage for protests")

for each in most_protests:
  if (each[0]) == 'USA':
    usa_amount = each
  print(f"* {each[0]} at {round(each[1],3)}% of total articles")

print(usa_amount)
dbg(protests)
all_more_than_usa = protests.filter(lambda x: x[1] >= usa_amount[1]).map(lambda x: x[0]).collect()

print (all_more_than_usa)



[(('CAN', '-1.3'), ['passengers', 'statement', 'pandemic', 'cancelled', 'air', 'passenger', 'refunds', 'coronavirus', 'airlines', 'agency', 'credit', 'fight', 'online', 'cta', 'canadian']), (('CAN', '-4.4'), ['guide', 'blind', 'covid19', 'account', 'develop', 'dont', 'rely', 'cnib', 'need', 'distancing', 'guides', 'shopping', 'nemeth', 'measures', 'sighted']), (('CAN', '-1.3'), ['passengers', 'statement', 'pandemic', 'cancelled', 'air', 'passenger', 'refunds', 'coronavirus', 'airlines', 'agency', 'credit', 'fight', 'online', 'cta', 'canadian']), (('CAN', '-2.9'), ['alleges', 'contract', 'union', 'issue', 'continues', 'dispute', 'city', 'meeting', 'resolution', 'residency', 'rule', 'breach', 'ontario', 'special', 'council']), (('CAN', '-10.'), ['victim', 'identify', 'clair', 'west', 'woundsthe', 'identified', 'witnesses', 'shooting', 'area', 'oakwood', 'man', 'st', None, None, None]), (('CAN', '-4.9'), ['street', 'emergency', 'free', 'closing', 'closes', 'pedestrians', 'city', 'cyclists

# Clustering

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler



def flatten(x):
  return [x[0]] + x[1]

flat_map = adjustedarray.map(lambda x: flatten(x))


headers = [str(i) for i in range(8)]
print(headers)

df = sqlContext.createDataFrame(flat_map,['COUNTRY'] +  headers)
df.show()
vecAssembler = VectorAssembler(inputCols=headers, outputCol="features")
new_df = vecAssembler.transform(df)

kmeans = KMeans(k=10, seed = 5) # ten clusters is sufficient for data. Expect 144 ^ 2 - 144
model = kmeans.fit(new_df.select('features'))

transformed = model.transform(new_df)
transformed.show()   



a = transformed.select('COUNTRY','prediction')
clusters = a.rdd.map(lambda x: list(x)).map(lambda x: (x[1],x[0])).groupByKey().collect()
 

usa_cluster = transformed.select('prediction').where("COUNTRY = 'USA'").rdd.map(lambda x: list(x)).map(lambda x: x).collect()[0]

print(usa_cluster)
all_usa_cluster = None
for each in clusters:
  if each[0] == usa_cluster[0]:
    all_usa_cluster = (list(each[1]))
    
  print(f"* Cluster: {each[0]} are {list(each[1])}")

print(all_usa_cluster)



['0', '1', '2', '3', '4', '5', '6', '7']
+-------+---+-------------------+-------------------+-------------------+-------------------+---+---+----+
|COUNTRY|  0|                  1|                  2|                  3|                  4|  5|  6|   7|
+-------+---+-------------------+-------------------+-------------------+-------------------+---+---+----+
|    CAN|0.0|0.20880991222872944| 0.4027048307268354| 0.6338872335515001| 0.6264297366861883|0.0|0.0|-0.0|
|    CHN|0.0|  0.491726111842046| 0.3226952608963427|   0.50709255283711| 0.6300240807976214|0.0|0.0|-0.0|
|    COL|0.0|0.08481889296799709| 0.4240944648399855| 0.5937322507759797| 0.6785511437439767|0.0|0.0|-0.0|
|    CUB|0.0| 0.6432675209026769| 0.5360562674188974| 0.5360562674188974|0.10721125348377948|0.0|0.0|-0.0|
|    DEU|0.0|0.16796775328675628|0.33593550657351257| 0.8398387664337815|  0.391924757669098|0.0|0.0|-0.0|
|    DNK|0.0|                0.0| 0.2672612419124244| 0.5345224838248488| 0.8017837257372732|0.0|0.0|-0

**RECCOMMENDATION**

In [ ]:
worse_than_us = ['CAN', 'CHN', 'GBR', 'IND', 'ITA', 'JOR', 'USA', 'NZL', 'TTO', 'VAT', 'JPN', 'KOR', 'RUS', 'VNM', 'DEU', 'COL']
reccommendation = adjustedarray.filter(lambda x: x[0] not in worse_than_us).filter(lambda x: x[0] not in all_more_than_usa).filter(lambda x: x[0]  in all_usa_cluster).sortBy(lambda x: cos_similarity(np.array(x[1]),USA_VEC)).collect()

print("Reccommend that the US aligns itself with these countries: Specifically such that the protest is removed:")


for i in range(0, len(reccommendation)):
  cand = reccommendation[i]
  print (cand[0] + ":")
  print([(j,round(float(k),2)) for j,k in zip(keys,cand[1])])
  print([(j,round(float(k),2)) for j,k in zip(keys,USA_VEC[1])])
  difference = [abs(USA_VEC[i] - cand[1][i]) for i in range(len(cand[1]))]
  print("DIFFERENCE NEEDED")
  print([(j,round(float(k),2)) for j,k in zip(keys,difference)])







Reccommend that the US aligns itself with these countries: Specifically such that the protest is removed:
LTU:
[('healthcare', 0.0), ('laws', 0.0), ('politics', 0.0), ('police', 0.71), ('economy', 0.71), ('education', 0.0), ('international_relations', 0.0), ('general_public_interest', -0.0)]
[('healthcare', 0.0), ('laws', 0.0), ('politics', 0.0), ('police', 0.71), ('economy', 0.71), ('education', 0.0), ('international_relations', 0.0), ('general_public_interest', -0.0)]
DIFFERENCE NEEDED
[('healthcare', 0.0), ('laws', 0.25), ('politics', 0.39), ('police', 0.03), ('economy', 0.14), ('education', 0.0), ('international_relations', 0.0), ('general_public_interest', 0.0)]
GHA:
[('healthcare', 0.0), ('laws', 0.0), ('politics', 0.0), ('police', 0.81), ('economy', 0.59), ('education', 0.0), ('international_relations', 0.0), ('general_public_interest', -0.0)]
[('healthcare', 0.0), ('laws', 0.0), ('politics', 0.0), ('police', 0.81), ('economy', 0.59), ('education', 0.0), ('international_relation

**RANDOM INPUT GENERATOR**

This is the implementation used on google cloud for testing. It can be scaled with the two global values at the beginning. 


As the implementation is severely limited by country code, we can have n entries instead of 144 max.


This part 'takes off' after the keywords and scraping has been done and we have our raw association vectors.

In [ ]:
#create: 
ENTRIES = 10000
CLUSTERS = 3
import random
from pyspark.sql import SQLContext
import numpy as np
import math
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler
sqlContext = SQLContext(sc)


def format_row(x):
  random_number = random.randint(0,16777215)
  hex_number = str(hex(random_number))
  row = [random.uniform(-1,1) for i in range(8)]
  return (str(x),row)



all_new_data = sc.parallelize([i for i in range(ENTRIES)]).map(format_row)


#make up the USA

down = all_new_data.take(1)
psuedo_us = down[0][0]
psuedo_vec = down[0][1]



def cos_similarity(v1, v2):
  return v1.dot(v2) / (np.linalg.norm(v1)*np.linalg.norm(v2))
b = all_new_data.map(lambda x: (x[0],np.array(x[1])))
c = b.map(lambda x: (x[0],cos_similarity(x[1], psuedo_vec),x[1]))
top_similar = c.takeOrdered(6, key = lambda x: 1-x[1])


for n in top_similar:
  pass

e = b.map(lambda x: (x[0],cos_similarity(x[1][1:-1], psuedo_vec[1:-1]),x[1]))
top_similar_just_media = e.takeOrdered(6, key = lambda x: 1-x[1])


for n in top_similar_just_media:
    pass


def simple_distance(x1,x2):
  return abs(x1 - x2)
d = b.map(lambda x: (x[0],simple_distance(x[1][-1], psuedo_vec[-1]),x[1]))
top_similar_tone = d.takeOrdered(6, key = lambda x: x[1])

for n in top_similar_tone:
  pass
 
top_similar_tone = d.takeOrdered(6, key = lambda x: 1-x[1])

for n in top_similar_tone:
  pass

# we need to generate some countries with worse better unrest than the US. For the purpose here, we will take a consistent sample of the data and just use those headers.
all_more_protest_than_us = all_new_data.takeSample(False, ENTRIES//10)
better_us = [i[0] for i in all_more_protest_than_us]

# PSUEDO. The real implementation is a simple filter, at worst O(n)

#clustering

def flatten(x):
  return [x[0]] + x[1]

flat_map = all_new_data.map(lambda x: flatten(x))


headers = [str(i) for i in range(8)]


df = sqlContext.createDataFrame(flat_map,['COUNTRY'] +  headers)
df.show()
vecAssembler = VectorAssembler(inputCols=headers, outputCol="features")
new_df = vecAssembler.transform(df)

kmeans = KMeans(k=CLUSTERS, seed = 5) # ten clusters is sufficient for data. Expect 144 ^ 2 - 144
model = kmeans.fit(new_df.select('features'))

transformed = model.transform(new_df)
transformed.show()   



a = transformed.select('COUNTRY','prediction')
clusters = a.rdd.map(lambda x: list(x)).map(lambda x: (x[1],x[0])).groupByKey().collect()


usa_cluster = transformed.select('prediction').where("COUNTRY = " + str(psuedo_us)).rdd.map(lambda x: list(x)).map(lambda x: x).collect()[0]

print(usa_cluster)

#.map(lambda x: x).collect()

all_usa_cluster = None
for each in clusters:
  if each[0] == usa_cluster[0]:
    all_usa_cluster = (list(each[1]))
    





# finally reccommendation:


worse_than_us = [i for i in range(0,ENTRIES//3)]
reccommendation = all_new_data.filter(lambda x: x[0] not in worse_than_us).filter(lambda x: x[0] not in worse_than_us).filter(lambda x: x[0]  in all_usa_cluster).sortBy(lambda x: cos_similarity(np.array(x[1]),psuedo_vec)).collect()
keys = ['healthcare','laws','politics','police','economy','education','international_relations','general_public_interest']



for i in range(0, len(reccommendation)):
  cand = reccommendation[i]

  difference = [abs(psuedo_vec[i] - cand[1][i]) for i in range(len(cand[1]))]



time_end = time.time()
print("elapsed time is %s" % str(time_end-time_start))





ModuleNotFoundError: ignored